### Función electric_potential

La función `electric_potential` resuelve el problema de valores en la frontera para el potencial eléctrico en un capacitor con dos placas no concéntricas. Utiliza el método de diferencias finitas en un sistema de coordenadas bipolares.

**Entradas:**
- `n_u1`: Número de puntos discretos en la dirección \( \sigma \).
- `n_u2`: Número de puntos discretos en la dirección \( \tau \).
- `v_ext`: Función que define el potencial eléctrico en la placa exterior.
- `v_int`: Potencial eléctrico en la placa interior.

**Salida:**
- Una matriz con tres columnas, donde cada fila representa las coordenadas \(i, j\) de un nodo y su correspondiente potencial eléctrico \( V \).

La función discretiza el dominio usando una malla regular en coordenadas \( \sigma \) y \( \tau \) y resuelve el sistema resultante para obtener el potencial en cada nodo.


In [ ]:
def electric_potential_corrected(n_u1, n_u2, v_ext, v_int):
    # Definir malla en coordenadas bipolares
    sigma = np.linspace(-np.pi, np.pi, n_u1)
    tau = np.linspace(0.9, 1.0, n_u2)
    d_sigma = sigma[1] - sigma[0]
    d_tau = tau[1] - tau[0]
    
    # Construir matrices de diferencias finitas
    A = csr_matrix((n_u1 * n_u2, n_u1 * n_u2))
    B = np.zeros(n_u1 * n_u2)
    
    for j in range(n_u2):
        for i in range(n_u1):
            # Coeficiente de ponderación
            coef = (np.cosh(tau[j]) - np.cosh(sigma[i]))**2
            
            # Index in the matrix A
            index = j * n_u1 + i
            
            # Center point
            center_coeff = -2 * coef / d_sigma**2 - 2 * coef / d_tau**2
            A[index, index] = center_coeff
            
            # Left point
            if i > 0:
                A[index, index - 1] = coef / d_sigma**2
            else:
                B[index] -= v_ext(sigma[i]) * coef / d_sigma**2
                
            # Right point
            if i < n_u1 - 1:
                A[index, index + 1] = coef / d_sigma**2
            else:
                B[index] -= v_ext(sigma[i]) * coef / d_sigma**2
                
            # Bottom point
            if j > 0:
                A[index, index - n_u1] = coef / d_tau**2
            else:
                B[index] -= v_ext(sigma[i]) * coef / d_tau**2
                
            # Top point
            if j < n_u2 - 1:
                A[index, index + n_u1] = coef / d_tau**2
            else:
                B[index] -= v_int * coef / d_tau**2
                
    # Resolver el sistema lineal
    V = spsolve(A, B)
    
    # Organizar resultados en matriz
    potential_matrix = np.zeros((n_u2, n_u1))
    for j in range(n_u2):
        for i in range(n_u1):
            index = j * n_u1 + i
            potential_matrix[j, i] = V[index]
            
    # Crear lista de resultados
    result = []
    for j in range(n_u2):
        for i in range(n_u1):
            result.append([i, j, potential_matrix[j, i]])
    
    return np.array(result)

# Probamos la función electric_potential_corrected
potencial_V_corrected = electric_potential_corrected(n_u1_test, n_u2_test, v_ext_test, v_int_test)
potencial_V_corrected


### Función electric_field

La función `electric_field` calcula el campo eléctrico a partir del potencial eléctrico obtenido previamente. Utiliza diferencias finitas centradas para calcular las derivadas del potencial y, por lo tanto, el campo eléctrico.

**Entradas:**
- `n_u1`: Número de puntos discretos en la dirección \( \sigma \).
- `n_u2`: Número de puntos discretos en la dirección \( \tau \).
- `potential_data`: Datos del potencial eléctrico obtenidos de la función `electric_potential`.

**Salida:**
- Una matriz con cuatro columnas, donde cada fila representa las coordenadas \(i, j\) de un nodo y las componentes del campo eléctrico \( E_{u1} \) y \( E_{u2} \).

El campo eléctrico se calcula como el negativo del gradiente del potencial eléctrico.


In [ ]:
def electric_field_corrected(n_u1, n_u2, potential_data):
    d_sigma = 2 * np.pi / (n_u1 - 1)
    d_tau = 0.1 / (n_u2 - 1)
    
    # Organizar potencial en matriz para facilitar cálculo de gradientes
    potential_matrix = np.zeros((n_u2, n_u1))
    for data in potential_data:
        i, j, V = data
        potential_matrix[int(j), int(i)] = V
    
    # Calcular derivadas discretas del potencial usando diferencias finitas centradas
    dV_du1 = np.zeros_like(potential_matrix)
    dV_du2 = np.zeros_like(potential_matrix)
    
    for j in range(1, n_u2-1):
        for i in range(1, n_u1-1):
            dV_du1[j, i] = (potential_matrix[j, i+1] - potential_matrix[j, i-1]) / (2 * d_sigma)
            dV_du2[j, i] = (potential_matrix[j+1, i] - potential_matrix[j-1, i]) / (2 * d_tau)
                
    # El campo eléctrico es el negativo del gradiente del potencial
    E_u1 = -dV_du1
    E_u2 = -dV_du2
    
    # Crear lista de resultados
    result = []
    for j in range(n_u2):
        for i in range(n_u1):
            result.append([i, j, E_u1[j, i], E_u2[j, i]])
    
    return np.array(result)

# Probamos nuevamente la función electric_field_corrected
compo_E_corrected = electric_field_corrected(n_u1_test, n_u2_test, potencial_V_corrected)
compo_E_corrected


### Función plot_electric_solution

La función `plot_electric_solution` visualiza el potencial eléctrico y la norma del campo eléctrico en todo el dominio \( \Omega \) en coordenadas polares.

**Entradas:**
- `n_u1`: Número de puntos discretos en la dirección \( \sigma \).
- `n_u2`: Número de puntos discretos en la dirección \( \tau \).
- `potential_data`: Datos del potencial eléctrico obtenidos de la función `electric_potential`.
- `electric_field_data`: Datos del campo eléctrico obtenidos de la función `electric_field`.

**Salida:**
- Dos gráficas: la primera muestra el potencial eléctrico y la segunda la norma del campo eléctrico en el dominio.

La función transforma los datos a coordenadas polares y utiliza un mapa de calor para visualizar los resultados.


In [ ]:
def plot_electric_solution(n_u1, n_u2, potential_data, electric_field_data):
    # Organizar datos en matrices para visualización
    potential_matrix = np.zeros((n_u2, n_u1))
    E_u1_matrix = np.zeros((n_u2, n_u1))
    E_u2_matrix = np.zeros((n_u2, n_u1))
    
    for data in potential_data:
        i, j, V = data
        potential_matrix[int(j), int(i)] = V
        
    for data in electric_field_data:
        i, j, E_u1, E_u2 = data
        E_u1_matrix[int(j), int(i)] = E_u1
        E_u2_matrix[int(j), int(i)] = E_u2
        
    E_norm_matrix = np.sqrt(E_u1_matrix**2 + E_u2_matrix**2)
    
    # Graficar potencial eléctrico
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(potential_matrix, origin="lower", extent=[-np.pi, np.pi, 0.9, 1], aspect='auto')
    plt.colorbar(label="Potencial eléctrico (V)")
    plt.title("Potencial eléctrico")
    plt.xlabel("σ")
    plt.ylabel("τ")
    
    # Graficar norma del campo eléctrico
    plt.subplot(1, 2, 2)
    plt.imshow(E_norm_matrix, origin="lower", extent=[-np.pi, np.pi, 0.9, 1], aspect='auto')
    plt.colorbar(label="Norma del campo eléctrico (|E|)")
    plt.title("Norma del campo eléctrico")
    plt.xlabel("σ")
    plt.ylabel("τ")
    
    plt.tight_layout()
    plt.show()

# Probamos la función plot_electric_solution
plot_electric_solution(n_u1_test, n_u2_test, potencial_V_test, compo_E_test)


### Función plot_electric_solution_cartesian

La función `plot_electric_solution_cartesian` visualiza el potencial eléctrico y la norma del campo eléctrico en todo el dominio \( \Omega \) en coordenadas cartesianas.

**Entradas:**
- `n_u1`: Número de puntos discretos en la dirección \( \sigma \).
- `n_u2`: Número de puntos discretos en la dirección \( \tau \).
- `potential_data`: Datos del potencial eléctrico obtenidos de la función `electric_potential`.
- `electric_field_data`: Datos del campo eléctrico obtenidos de la función `electric_field`.

**Salida:**
- Dos gráficas: la primera muestra el potencial eléctrico y la segunda la norma del campo eléctrico en el dominio.

La función transforma los datos a coordenadas cartesianas usando la relación entre las coordenadas bipolares y cartesianas y utiliza un mapa de calor para visualizar los resultados.


In [ ]:
def plot_electric_solution_cartesian(n_u1, n_u2, potential_data, electric_field_data):
    # Organizar datos en matrices para visualización
    potential_matrix = np.zeros((n_u2, n_u1))
    E_u1_matrix = np.zeros((n_u2, n_u1))
    E_u2_matrix = np.zeros((n_u2, n_u1))
    
    for data in potential_data:
        i, j, V = data
        potential_matrix[int(j), int(i)] = V
        
    for data in electric_field_data:
        i, j, E_u1, E_u2 = data
        E_u1_matrix[int(j), int(i)] = E_u1
        E_u2_matrix[int(j), int(i)] = E_u2
        
    E_norm_matrix = np.sqrt(E_u1_matrix**2 + E_u2_matrix**2)
    
    # Coordenadas bipolares
    sigma = np.linspace(-np.pi, np.pi, n_u1)
    tau = np.linspace(0.9, 1.0, n_u2)
    
    # Transformación a coordenadas cartesianas
    X, Y = np.meshgrid(sigma, tau)
    x = Y * np.cos(X) / (np.cosh(Y) - np.cos(X))
    y = Y * np.sin(X) / (np.cosh(Y) - np.cos(X))
    
    # Graficar potencial eléctrico en coordenadas cartesianas
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.pcolormesh(x, y, potential_matrix, shading='auto')
    plt.colorbar(label="Potencial eléctrico (V)")
    plt.title("Potencial eléctrico")
    plt.xlabel("x")
    plt.ylabel("y")
    
    # Graficar norma del campo eléctrico en coordenadas cartesianas
    plt.subplot(1, 2, 2)
    plt.pcolormesh(x, y, E_norm_matrix, shading='auto')
    plt.colorbar(label="Norma del campo eléctrico (|E|)")
    plt.title("Norma del campo eléctrico")
    plt.xlabel("x")
    plt.ylabel("y")
    
    plt.tight_layout()
    plt.show()

# Probamos nuevamente la función plot_electric_solution_cartesian con los datos corregidos
plot_electric_solution_cartesian(n_u1_test, n_u2_test, potencial_V_corrected, compo_E_corrected)


### Función plot_electric_solution_polar

La función `plot_electric_solution_polar` visualiza el potencial eléctrico y la norma del campo eléctrico en todo el dominio \( \Omega \) en coordenadas polares.

**Entradas:**
- `n_u1`: Número de puntos discretos en la dirección \( \sigma \).
- `n_u2`: Número de puntos discretos en la dirección \( \tau \).
- `potential_data`: Datos del potencial eléctrico obtenidos de la función `electric_potential`.
- `electric_field_data`: Datos del campo eléctrico obtenidos de la función `electric_field`.

**Salida:**
- Dos gráficas en coordenadas polares: la primera muestra el potencial eléctrico y la segunda la norma del campo eléctrico en el dominio.

La función convierte las coordenadas cartesianas a polares y utiliza un mapa de calor en un gráfico polar para visualizar los resultados.


In [ ]:
def plot_electric_solution_polar(n_u1, n_u2, potential_data, electric_field_data):
    # Organizar datos en matrices para visualización
    potential_matrix = np.zeros((n_u2, n_u1))
    E_u1_matrix = np.zeros((n_u2, n_u1))
    E_u2_matrix = np.zeros((n_u2, n_u1))
    
    for data in potential_data:
        i, j, V = data
        potential_matrix[int(j), int(i)] = V
        
    for data in electric_field_data:
        i, j, E_u1, E_u2 = data
        E_u1_matrix[int(j), int(i)] = E_u1
        E_u2_matrix[int(j), int(i)] = E_u2
        
    E_norm_matrix = np.sqrt(E_u1_matrix**2 + E_u2_matrix**2)
    
    # Coordenadas bipolares
    sigma = np.linspace(-np.pi, np.pi, n_u1)
    tau = np.linspace(0.9, 1.0, n_u2)
    
    # Transformación a coordenadas cartesianas
    X, Y = np.meshgrid(sigma, tau)
    x = Y * np.cos(X) / (np.cosh(Y) - np.cos(X))
    y = Y * np.sin(X) / (np.cosh(Y) - np.cos(X))
    
    # Convertir a coordenadas polares
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    
    # Graficar potencial eléctrico en coordenadas polares
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1, projection='polar')
    plt.pcolormesh(theta, r, potential_matrix, shading='auto')
    plt.colorbar(label="Potencial eléctrico (V)")
    plt.title("Potencial eléctrico")
    
    # Graficar norma del campo eléctrico en coordenadas polares
    plt.subplot(1, 2, 2, projection='polar')
    plt.pcolormesh(theta, r, E_norm_matrix, shading='auto')
    plt.colorbar(label="Norma del campo eléctrico (|E|)")
    plt.title("Norma del campo eléctrico")
    
    plt.tight_layout()
    plt.show()

# Probamos la función plot_electric_solution_polar con los datos corregidos
plot_electric_solution_polar(n_u1_test, n_u2_test, potencial_V_corrected, compo_E_corrected)
